<a href="https://colab.research.google.com/github/SamuelSousaFerreira/pyspark/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install depencences
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # java instalattion
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz # file's download
!tar xf spark-3.1.2-bin-hadoop2.7.tgz #file descompression
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

# **Upload Data**
## [SparkSession](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.html)

O ponto de entrada para programar o Spark com a API Dataset e DataFrame.

Uma SparkSession pode ser utilizada para criar DataFrames, registrar DataFrames como tabelas, executar consultas SQL em tabelas, armazenar em cache e ler arquivos parquet. Para criar uma SparkSession, use o seguinte padrão de construtor:

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = (SparkSession.builder
         .master('local[*]')
         .appName("Iniciando com Spark")
         .getOrCreate()    
        )

In [6]:
spark

In [7]:
spark = (SparkSession.builder
         .master('local[*]')
         .appName("Iniciando com Spark")
         .config('spark.ui.port', '4050')
         .getOrCreate()    
        )

In [8]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [9]:
get_ipython().system_raw('./ngrok authtoken 254WBMksot5DG6U7eUPV40u9NO9_7Q5YgfApAgwsd6bdZ8GHy')
get_ipython().system_raw('./ngrok http 4050 &')

In [10]:
!curl -s http://localhost:4040/api/tunnels

<html>
<head>
<meta http-equiv="Content-Type" content="text/html;charset=utf-8"/>
<title>Error 404 Not Found</title>
</head>
<body><h2>HTTP ERROR 404 Not Found</h2>
<table>
<tr><th>URI:</th><td>/api/tunnels</td></tr>
<tr><th>STATUS:</th><td>404</td></tr>
<tr><th>MESSAGE:</th><td>Not Found</td></tr>
<tr><th>SERVLET:</th><td>org.glassfish.jersey.servlet.ServletContainer-3b76be5a</td></tr>
</table>
<hr><a href="https://eclipse.org/jetty">Powered by Jetty:// 9.4.40.v20210413</a><hr/>

</body>
</html>


In [11]:
spark

# **Making a DataFrame**

In [12]:
data = [('Zeca','35'), ('Eva', '29')]
colNames = ["Nome",'Idade']

In [13]:
df = spark.createDataFrame(data, colNames)
df

DataFrame[Nome: string, Idade: string]

In [14]:
df.show()

+----+-----+
|Nome|Idade|
+----+-----+
|Zeca|   35|
| Eva|   29|
+----+-----+



In [15]:
df.toPandas()

,Nome,Idade
0,Zeca,35
1,Eva,29


# **Projeto**

## Carregando os dados das empresas

In [16]:
import zipfile

In [17]:
zipfile.ZipFile('/content/drive/MyDrive/Cursos/pyspark/Aulas_de_spark/dados/empresas.zip','r').extractall('/content/drive/MyDrive/Cursos/pyspark/Aulas_de_spark/dados')

In [18]:
path = '/content/drive/MyDrive/Cursos/pyspark/Aulas_de_spark/dados/empresas'

In [19]:
empresas = spark.read.csv(path, sep =';',inferSchema=True)

In [20]:
empresas.count()

4585679

In [21]:
empresas.show()

+-----+--------------------+----+---+-------+---+----+
|  _c0|                 _c1| _c2|_c3|    _c4|_c5| _c6|
+-----+--------------------+----+---+-------+---+----+
|  306|FRANCAMAR REFRIGE...|2240| 49|   0,00|  1|null|
| 1355|BRASILEIRO & OLIV...|2062| 49|   0,00|  5|null|
| 4820|REGISTRO DE IMOVE...|3034| 32|   0,00|  5|null|
| 5347|ROSELY APARECIDA ...|2135| 50|   0,00|  5|null|
| 6846|BADU E FILHOS TEC...|2062| 49|4000,00|  1|null|
| 8416|  ELETRICA RUBI LTDA|2062| 49|   0,00|  5|null|
| 8992|SHIROMA VEICULOS ...|2062| 49|   0,00|  5|null|
| 9091|CONTATOS BAR E LA...|2062| 49|   0,00|  5|null|
| 9614|ANTONIA APARECIDA...|2135| 50|   0,00|  5|null|
| 9896|DORACY CORAT DA C...|2135| 50|   0,00|  5|null|
|12112|LANCHONETE RIO VE...|2062| 49|   0,00|  5|null|
|12605|VALMAR JACAREI CO...|2062| 49|   0,00|  5|null|
|13407|ROSANA CRISTINA D...|2135| 50|   0,00|  5|null|
|13408|CELIO RODRIGUES D...|2135| 50|   0,00|  5|null|
|13721|MAQFRAN COMERCIO ...|2062| 49|   0,00|  1|null|
|21181|MOU

## Sócios

In [22]:
zipfile.ZipFile('/content/drive/MyDrive/Cursos/pyspark/Aulas_de_spark/dados/socios.zip').extractall('/content/drive/MyDrive/Cursos/pyspark/Aulas_de_spark/dados')

In [23]:
path_socios = "/content/drive/MyDrive/Cursos/pyspark/Aulas_de_spark/dados/socios/part-0000*" 
#* O caractere *, neste caso, está indicando que queremos ler todos os arquivos que tenham como início de seu nome “EMPRESA.SP”. Quaisquer caracteres que vierem após isso são válidos.

In [24]:
socios = spark.read.csv(path_socios,sep =';',inferSchema=True)

In [25]:
socios.count()

2046430

In [26]:
socios.show()

+-----+---+--------------------+-----------+---+--------+----+-----------+----+---+----+
|  _c0|_c1|                 _c2|        _c3|_c4|     _c5| _c6|        _c7| _c8|_c9|_c10|
+-----+---+--------------------+-----------+---+--------+----+-----------+----+---+----+
|  411|  2|LILIANA PATRICIA ...|***678188**| 22|19940725|null|***000000**|null|  0|   7|
|  411|  2|CRISTINA HUNDERTMARK|***637848**| 28|19940725|null|***000000**|null|  0|   7|
| 5813|  2|CELSO EDUARDO DE ...|***786068**| 49|19940516|null|***000000**|null|  0|   8|
| 5813|  2|EDUARDO BERRINGER...|***442348**| 49|19940516|null|***000000**|null|  0|   5|
|14798|  2| HANNE MAHFOUD FADEL|***760388**| 49|19940609|null|***000000**|null|  0|   8|
|14798|  2|    CLOD ASSAD FADEL|***205668**| 22|19940609|null|***000000**|null|  0|   6|
|17826|  2|   WALKYRIA ALGARVES|***689078**| 49|19970227|null|***000000**|null|  0|   7|
|17826|  2|SEBASTIAO JADIR T...|***904728**| 49|20090813|null|***000000**|null|  0|   5|
|19491|  2|     JOSE 

## Estabelecimentos

In [27]:
zipfile.ZipFile('/content/drive/MyDrive/Cursos/pyspark/Aulas_de_spark/dados/estabelecimentos.zip').extractall('/content/drive/MyDrive/Cursos/pyspark/Aulas_de_spark/dados')

In [28]:
path_estabelecimentos = '/content/drive/MyDrive/Cursos/pyspark/Aulas_de_spark/dados/estabelecimentos'

In [29]:
estabelecimentos  = spark.read.csv(path_estabelecimentos, sep=';',inferSchema=True)

In [30]:
estabelecimentos.count()

4836219

In [31]:
estabelecimentos.show()

+-----+---+---+---+--------------------+---+--------+---+----+----+--------+-------+--------------------+-------+--------------------+------+-------------------+------------------+--------+----+----+----+--------+----+--------+----+--------+--------------------+----+----+
|  _c0|_c1|_c2|_c3|                 _c4|_c5|     _c6|_c7| _c8| _c9|    _c10|   _c11|                _c12|   _c13|                _c14|  _c15|               _c16|              _c17|    _c18|_c19|_c20|_c21|    _c22|_c23|    _c24|_c25|    _c26|                _c27|_c28|_c29|
+-----+---+---+---+--------------------+---+--------+---+----+----+--------+-------+--------------------+-------+--------------------+------+-------------------+------------------+--------+----+----+----+--------+----+--------+----+--------+--------------------+----+----+
| 1879|  1| 96|  1|      PIRAMIDE M. C.|  8|20011029|  1|null|null|19940509|1412602|                null|    RUA|     JOSE FIGLIOLINI|   608|               null|         VILA NILO| 

# **Manipulando os dados**

## Operações básicas

In [32]:
empresas.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


## Renomeando colunas



In [33]:
empresasColNames = ['cnpj_basico','razao_social_nome_empresarial','naureza_juridica','qualificacao_do responsavel',
                    'capital_social_da_empresa','porte_da_empresa','ente_federativo_responsavel']

In [34]:
for index, colName in enumerate(empresasColNames):
  empresas = empresas.withColumnRenamed(f"_c{index}", colName)
empresas.columns

['cnpj_basico',
 'razao_social_nome_empresarial',
 'naureza_juridica',
 'qualificacao_do responsavel',
 'capital_social_da_empresa',
 'porte_da_empresa',
 'ente_federativo_responsavel']

In [35]:
estabsColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 
                  'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 
                  'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']

sociosColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 
                  'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [36]:
for index, colName in enumerate(estabsColNames):
  
  estabelecimentos = estabelecimentos.withColumnRenamed(f"_c{index}", colName)
estabelecimentos.columns


['cnpj_basico',
 'cnpj_ordem',
 'cnpj_dv',
 'identificador_matriz_filial',
 'nome_fantasia',
 'situacao_cadastral',
 'data_situacao_cadastral',
 'motivo_situacao_cadastral',
 'nome_da_cidade_no_exterior',
 'pais',
 'data_de_inicio_atividade',
 'cnae_fiscal_principal',
 'cnae_fiscal_secundaria',
 'tipo_de_logradouro',
 'logradouro',
 'numero',
 'complemento',
 'bairro',
 'cep',
 'uf',
 'municipio',
 'ddd_1',
 'telefone_1',
 'ddd_2',
 'telefone_2',
 'ddd_do_fax',
 'fax',
 'correio_eletronico',
 'situacao_especial',
 'data_da_situacao_especial']

In [37]:
for index, colName in enumerate(sociosColNames):
  
  socios = socios.withColumnRenamed(f"_c{index}", colName)
socios.columns

['cnpj_basico',
 'identificador_de_socio',
 'nome_do_socio_ou_razao_social',
 'cnpj_ou_cpf_do_socio',
 'qualificacao_do_socio',
 'data_de_entrada_sociedade',
 'pais',
 'representante_legal',
 'nome_do_representante',
 'qualificacao_do_representante_legal',
 'faixa_etaria']

## Analisando os dados

In [38]:
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,naureza_juridica,qualificacao_do responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [39]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- naureza_juridica: integer (nullable = true)
 |-- qualificacao_do responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [40]:
socios.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,19940725,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,19940725,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,19940516,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,19940516,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,19940609,NaN,***000000**,None,0,8


In [41]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: integer (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [42]:
estabelecimentos.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,data_de_inicio_atividade,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_de_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,1879,1,96,1,PIRAMIDE M. C.,8,20011029,1,None,NaN,19940509,1412602,None,RUA,JOSE FIGLIOLINI,608,None,VILA NILO,2278020,SP,7107,None,None,None,None,NaN,None,None,None,NaN
1,2818,1,43,1,None,8,20081231,71,None,NaN,19940512,4671100,None,RUA,BAQUIA,416,None,VL NOVA MANCHESTER,3443000,SP,7107,None,None,None,None,NaN,None,None,None,NaN
2,3110,1,7,1,None,8,19971231,1,None,NaN,19940512,4789007,None,RUA,LEOCADIA CINTRA,180,CONJ 83,MOOCA,3112040,SP,7107,None,None,None,None,NaN,None,None,None,NaN
3,3733,1,80,1,None,8,20081231,71,None,NaN,19940513,7490101,None,RUA,AFONSO CELSO,1102,CASA 1,VILA MARIANA,4119061,SP,7107,None,None,None,None,NaN,None,None,None,NaN
4,4628,3,27,2,EMBROIDERY & GIFT,8,19980429,1,None,NaN,19950509,4755501,None,RUA,DOUTOR GABRIEL NICOLAU,177,None,RUDGE RAMOS,9632040,SP,7075,None,None,None,None,NaN,None,None,None,NaN


In [45]:
estabelecimentos.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: integer (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: integer (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: str

# Modificando os tipos de dados

## Convertendo string para Double type


In [47]:
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as F

In [48]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- naureza_juridica: integer (nullable = true)
 |-- qualificacao_do responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [52]:
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,naureza_juridica,qualificacao_do responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.00,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.00,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.00,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.00,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.00,1,None


In [53]:
empresas = empresas.withColumn('capital_social_da_empresa', F.regexp_replace('capital_social_da_empresa',',','.'))
#quando for substituir o ponto deve se usar uma barra invertida antes '\.'
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,naureza_juridica,qualificacao_do responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.00,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.00,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.00,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.00,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.00,1,None


In [54]:
empresas.printSchema() #capital social continua como string mesmo depois de substituir a vírgula pelo ponto

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- naureza_juridica: integer (nullable = true)
 |-- qualificacao_do responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [61]:
empresas = empresas.withColumn('capital_social_da_empresa',empresas['capital_social_da_empresa'].cast(DoubleType()))
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,naureza_juridica,qualificacao_do responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.0,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.0,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.0,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.0,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.0,1,None


In [62]:
empresas.printSchema() 

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- naureza_juridica: integer (nullable = true)
 |-- qualificacao_do responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)

